# Simulación de un Sistema de Transporte Público Interurbano

### **Objetivo de la Simulación:**

**Simular el Funcionamiento del Sistema de Transporte Público Interurbano**  
El objetivo de esta simulación es modelar el sistema de transporte público entre varias ciudades para observar y analizar su comportamiento bajo diferentes condiciones. La simulación permitirá estudiar cómo varía el tiempo de espera de los pasajeros, la utilización de los vehículos, y la eficiencia general del sistema en función de variables como la demanda de pasajeros, la frecuencia de los servicios, y la capacidad de los vehículos. Además, se busca evaluar el impacto de eventos aleatorios, como retrasos y fluctuaciones en la demanda, para entender mejor el rendimiento del sistema en escenarios reales.

**Descripción del Problema de Colas:**

En esta simulación, el sistema de transporte público se modela como un problema de colas múltiples. Los pasajeros son los "clientes" que llegan a las estaciones (las "colas") y esperan a que un autobús o tren (el "servidor") llegue para transportarlos a su destino.

# Recursos


In [ ]:
import simpy
import pandas as pd
import random
import numpy as np
import ipdb

In [53]:


class Gasolinera:
    def __init__(self, env, name, capacity_fuel, capacity_bomb):
        self.env = env
        self.name = name
        self.capacity_fuel = capacity_fuel
        self.capacity_bomb = capacity_bomb
        self.capacidad_gasolina = simpy.Container(env, init=capacity_fuel, capacity=capacity_fuel)
        self.bombas = simpy.Resource(env, capacity=capacity_bomb)
        self.process = env.process(self.run())
        
        # DataFrames para registrar eventos
        self.df_repostajes = pd.DataFrame(columns=["time", "vehicle_name", "quantity", "event"])
        self.df_cargas = pd.DataFrame(columns=["time", "quantity", "event"])

    def run(self):
        """Proceso que gestiona el repostaje y la carga de gasolina en la gasolinera."""
        while True:
            yield self.env.timeout(1)  # Simular tiempo de operación

    def refuel_vehicle(self, cantidad, name):
        """Proceso para repostar gasolina a un vehículo."""
        with self.bombas.request() as request:
            yield request
            
            # Verificar si hay suficiente gasolina en la gasolinera
            if self.capacidad_gasolina.level >= cantidad:
                cantidad = round(cantidad, 2)
                # Simular el tiempo de repostaje
                time = round(cantidad * 0.1, 2)
                yield self.env.timeout(time)  # Tiempo de repostaje (0.5 min por unidad de gasolina)
                yield self.env.timeout(random.randint(4, 7))
                # Repostar gasolina al vehículo y reducir el nivel de gasolina en la gasolinera
                yield self.capacidad_gasolina.get(cantidad)
                
                print(f'Tiempo {self.env.now}: Repostando {cantidad} l de gasolina en el vehículo {name}.')
                
                # Registrar evento de repostaje
                self.df_repostajes = pd.concat([
                    self.df_repostajes, 
                    pd.DataFrame({"time": [self.env.now], "vehicle_name": [name], "quantity": [cantidad], "event": ["repostaje"]})
                ])
                
            else:
                print(f'Tiempo {self.env.now}: No hay suficiente gasolina en la gasolinera para repostar el vehículo {name}.')
                yield from self.load_gasoline()  

                # Registrar evento de falta de gasolina
                self.df_repostajes = pd.concat([
                    self.df_repostajes, 
                    pd.DataFrame({"time": [self.env.now], "vehicle_name": [name], "quantity": [0], "event": ["falta de gasolina"]})
                ])

    def load_gasoline(self):
        """Proceso para cargar gasolina en la gasolinera."""
        yield self.env.timeout(20)  # Tiempo para cargar gasolina (5 min en este ejemplo)
        # Asegurarse de que la gasolina cargada no exceda la capacidad
        cantidad_cargada = self.capacity_fuel - self.capacidad_gasolina.level
        
        print(f'Tiempo {self.env.now}: Cargada {cantidad_cargada} unidades de gasolina en la gasolinera.')
        yield self.capacidad_gasolina.put(cantidad_cargada)
        print(f'Tiempo {self.env.now}: Cargado {cantidad_cargada} unidades de gasolina en la gasolinera.')

        # Registrar evento de carga de gasolina
        self.df_cargas = pd.concat([
            self.df_cargas, 
            pd.DataFrame({"time": [self.env.now], "quantity": [cantidad_cargada], "event": ["carga de gasolina"]})
        ])


In [67]:
class Passenger:
    def __init__(self, env, name,origin,destination,vehicle):
        self.env=env
        self.name=name
        self.origin=origin
        self.destination=destination
        self.vehicle=vehicle
        self.state=False
        self.process = env.process(self.travel())
    """
        Controla el proceso de viaje del pasajero, que incluye esperar en la estación, 
        abordar el vehículo, viajar y salir del vehículo.
    """
    def travel(self):
        #esperar en la estacion
        yield from self.wait_in_station()
        #subirse al vehiculo
        yield from self.boardin_vehicle()
        if self.state:
            #viajar en el vehiculo
            yield from self.wait_in_travel()
            #salir del vehiculo
            yield from self.exit_vehicle() 

    """
        Hace que el pasajero espere en la estación hasta que el vehículo llegue al punto de origen.

        Se utiliza un bucle para esperar hasta que el vehículo esté en la estación de origen.
        Luego, se imprime el tiempo total de espera del pasajero.
    """
    def wait_in_station(self):
        t=self.env.now
        #ipdb.set_trace()
        while self.vehicle.current_route!=self.origin:
            yield self.env.timeout(1)

        print(f'Tiempo {self.env.now}: {self.name} espero por {self.env.now-t} min')
        yield self.env.timeout(1)

    """
        Intenta abordar al pasajero en el vehículo cuando está en la estación de origen.

        Usa un recurso de SimPy para gestionar el abordaje de pasajeros y actualiza 
        el estado del pasajero dependiendo de si logra abordar o no.
    """
    def boardin_vehicle(self):
        with self.vehicle.resource.request() as request:
            yield request
            print(f'Tiempo {self.env.now}: {self.name} intenta abordar en {self.origin}')
            self.state = yield self.env.process(self.vehicle.board_passenger(self.name))

                
                    
    """
        Hace que el pasajero espere mientras viaja hacia su destino.

        Verifica continuamente si el vehículo ha llegado a la estación de destino
        y calcula el tiempo total de viaje.
    """
    def wait_in_travel(self):
        time_start=self.env.now
        while self.vehicle.current_route!=self.destination:
            yield self.env.timeout(random.randint(1,2))
            
        print(f'Tiempo {self.env.now} pasajero {self.name} viajo por {self.env.now-time_start} min')
        yield self.env.timeout(1)
    """
        Maneja el proceso de salida del pasajero del vehículo al llegar a su destino.

        Imprime un mensaje indicando que el pasajero ha llegado a su destino y
        libera un espacio en el vehículo.
    """
    def exit_vehicle(self):
        print(f'Tiempo {self.env.now}: {self.name}: llego a {self.destination}')
        yield from self.vehicle.unboard_passenger()
        yield self.env.timeout(1)
        

In [187]:
class Vehicle:
    def __init__(self,env, name, route,capacity_passenger,capacity_tank,petrol_station):
                 self.env=env
                 self.name=name
                 self.route=route
                 self.capacity_tank=capacity_tank
                 self.capacity=simpy.Container( env , init=capacity_passenger , capacity=capacity_passenger  )
                 self.tank=simpy.Container( env , init=capacity_tank , capacity=capacity_tank  )
                 self.passenger=[]
                 self.current_route=route[0]
                 self.resource = simpy.Resource(env, capacity=capacity_passenger)
                 self.process = env.process(self.run_route())
                 self.petrol_station=petrol_station
                 
    def run_route(self):
     
        while True:
            #Autobus recorre estaciones
            wait_time=0
            for station in self.route:
                #vehiculo llega a la estacion 
                time=random.randint(10,15)
                #se utilza aproximada el 5% de tanque por minuto
                gasolina_necesaria = time * (5 / 100)
                
                #se verifica que exista gasolina suficiente
                if self.tank.level >= gasolina_necesaria:
                    
                    #si es la primera estacion entonces el vehiculo no tarda en llegar a esta
                    if self.route.index(station)!=0:
                           
                        yield self.env.timeout(time)
                        
                        print(f'tiempo {self.env.now}: El vehiculo ha llegado a la estacion {station} en {time} min')
                    else:
                        print(f'tiempo {self.env.now}: El vehiculo esta en la estacion  {station}')
                        time=0
                     #se extrae la gasolina
                    yield self.tank.get(gasolina_necesaria)
                    wait_time=self.env.now
                    self.current_route=station
                else:
                    #de lo contrario se verfica hasta donde pudo llegar el vehiculo
                    current_tank=self.tank.level
                    new_time=(current_tank*100)/5
                    yield self.tank.get(current_tank)
                    print(f'Tanque vacío a {round(time-new_time,5)} min de {station}. El vehículo no puede continuar.')
                    return
                #tiempo de espera anes de salir de la estacion 
                yield self.env.timeout(random.randint(3,7)) 
                #si el nivel de la gasolina es menor a 10 y se encuentra en una estacion de carga, entonces se carga el combustible
                if station in self.petrol_station.keys() and self.tank.level<=10:
                       yield from self.chargin_fuel(station)
                
                print(f'tiempo {self.env.now}: El vehiculo ha salido de la estacion {station} despues de {round(self.env.now-wait_time,2)} min')
            #se invierten las rutas
            self.route.reverse()
                
    def chargin_fuel(self,station):
         time_chargin=self.env.now
         cantidad=self.capacity_tank-self.tank.level
         print(f'Tiempo {self.env.now}: cobustible casi agotado en {station}')
        
         yield self.env.timeout(1)
         #ipdb.set_trace()
         try:
            yield self.env.process(self.petrol_station[station].refuel_vehicle( cantidad,self.name))
         except simpy.Interrupt as i:
            print( f"{self.id}: message: { i.cause }" )
         yield self.tank.put(cantidad)
         print(f'Tiempo {self.env.now}: Saliendo de estacion de combustible despues de {round(self.env.now-time_chargin)} min')
        
    def board_passenger(self, name):
        if self.capacity.level > 0:
            self.capacity.get(1)
            print(f'Tiempo {self.env.now}: {name} abordo en {self.current_route}')
            yield self.env.timeout(1)
            return True
        else:
            print(f'Tiempo {self.env.now}: vehiculo lleno')
            print(f'Tiempo {self.env.now} {name}: no pudo abordar')
            return False

        
    def unboard_passenger(self):
            self.capacity.put(1)
            yield self.env.timeout(1)
        



In [57]:

class Setup:
    def __init__(self, env, num_vehicles, num_passengers, route):
        self.env = env
        self.vehicles = []
        self.passengers = []
        self.route = route
        self.petrol_stations=dict()
        for station in np.random.choice(route,2,replace=False):
            self.petrol_stations[station]=Gasolinera(env,station,random.randint(300,400),random.randint(4,7))
            

        # Crear vehículos
        for i in range(num_vehicles):
            vehicle = Vehicle(
                env,
                f"Vehicle {i+1}",
                route,
                capacity_passenger=random.randint(20, 50),
                capacity_tank=random.randint(20, 40),
                petrol_station=self.petrol_stations
            )
            self.vehicles.append(vehicle)

        # Simular la llegada de pasajeros
        env.process(self.arriving_passengers(num_passengers))

    def arriving_passengers(self, num_passengers):
        for i in range(num_passengers):
            origin, destination = random.sample(self.route, 2)
            passenger = Passenger(
                self.env,
                f"Passenger {i+1}",
                origin,
                destination,
                random.choice(self.vehicles)
            )
            self.passengers.append(passenger)
            yield self.env.timeout(random.randint(1, 300))  # Llegada aleatoria de pasajeros


In [197]:
# Paso 1: Definir rutas y gasolineras
route = ['San Francisco', 'Uvas', 'El Loarque', 'La Cañada', 'Villas del Sol', 'El Trapiche', 'Altos del Trapiche', 'CU', 'Col. Suyapa']


env = simpy.Environment()


num_vehicles = 10
num_passengers = 20
setup = Setup(env, num_vehicles, num_passengers, route)

# Paso 4: Ejecutar la simulación
simulation_time = 100
env.run(until=simulation_time)

tiempo 0: El vehiculo esta en la estacion  San Francisco
tiempo 0: El vehiculo esta en la estacion  San Francisco
tiempo 0: El vehiculo esta en la estacion  San Francisco
tiempo 0: El vehiculo esta en la estacion  San Francisco
tiempo 0: El vehiculo esta en la estacion  San Francisco
tiempo 0: El vehiculo esta en la estacion  San Francisco
tiempo 0: El vehiculo esta en la estacion  San Francisco
tiempo 0: El vehiculo esta en la estacion  San Francisco
tiempo 0: El vehiculo esta en la estacion  San Francisco
tiempo 0: El vehiculo esta en la estacion  San Francisco
tiempo 3: El vehiculo ha salido de la estacion San Francisco despues de 3 min
tiempo 4: El vehiculo ha salido de la estacion San Francisco despues de 4 min
tiempo 4: El vehiculo ha salido de la estacion San Francisco despues de 4 min
tiempo 4: El vehiculo ha salido de la estacion San Francisco despues de 4 min
tiempo 5: El vehiculo ha salido de la estacion San Francisco despues de 5 min
tiempo 5: El vehiculo ha salido de la es